# fastcore.meta.delegates

## Imports

In [ ]:
from debuggable.utils import dbprint, defaults, whichversion, defaults, colorize, insert2debug, dbsrclines, checksrc

In [ ]:
# !pip install -U fastcore
whichversion('fastcore')

fastcore: 1.5.22 
Python supercharged for fastai development    
Jeremy Howard and Sylvain Gugger 
https://github.com/fastai/fastcore/     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore


In [ ]:
from fastcore.imports import *
from fastcore.meta import *
from fastcore.test import *
import inspect
from pprint import pprint
from datetime import date
import ast

In [ ]:
today = date.today()
print("Today's date:", today)

Today's date: 2022-08-26


## Upate my source

In [ ]:
srcname = "delegates"
checksrc(srcname)

def delegates(to:FunctionType=None, # Delegatee                                                                                                              
              keep=False, # Keep `kwargs` in decorated function?                                                                                             
              but:list=None): # Exclude these parameters from signature                                                                                      
    "Decorator: replace `**kwargs` in signature with params from `to`"                                                                                       
    if but is None: but = []                                                                                                                                 
    def _f(f):                                                                                                                                               
        if to is None: to_f,from_f = f.__base__.__in

## A personal docs on delegates

I have been developing a workflow of creating my own docs for fastcore. Here is what I have so far.

1st step: read and run examples from the official [docs](https://hyp.is/8brb_B42Ee2vbjuvJtjGGw/nbviewer.org/github/fastai/fastcore/blob/master/nbs/07_meta.ipynb) to understand how you should use it.

2nd step: write your own examples to test your understanding of the usages and explore the boundaries to cause errors.

3rd step: read the source and find out what from the source caused the errors using debugging tools like `print`, `pdb.set_trace`, `%debug`, jupyterlab's visual debugger etc.

To keep my docs here readable, I will only present the second step and one or two examples of the third step.

In [ ]:
pprint(inspect.getsource(delegates))

('def delegates(to:FunctionType=None, # Delegatee\n'
 '              keep=False, # Keep `kwargs` in decorated function?\n'
 '              but:list=None): # Exclude these parameters from signature\n'
 '    "Decorator: replace `**kwargs` in signature with params from `to`"\n'
 '    if but is None: but = []\n'
 '    def _f(f):\n'
 '        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\n'
 '        else:          to_f,from_f = to.__init__ if isinstance(to,type) else '
 'to,f\n'
 "        from_f = getattr(from_f,'__func__',from_f)\n"
 "        to_f = getattr(to_f,'__func__',to_f)\n"
 "        if hasattr(from_f,'__delwrap__'): return f\n"
 '        sig = inspect.signature(from_f)\n'
 '        sigd = dict(sig.parameters)\n'
 "        k = sigd.pop('kwargs')\n"
 '        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in '
 'inspect.signature(to_f).parameters.items()\n'
 '              if v.default != inspect.Parameter.empty and k not in sigd and '
 'k not in but}\

### When to use it?
When `f` want to replace its `**kwargs` param with unique kwargs (with their annotations) from `to`.

### Who are `to` and `f`?
They can be functions, instance methods, classmethods, and even classes.

### What is `**kwargs`?
It's a param of `f`, to receive unknown keyword args (i.e., not sure how many and what they are) with values. In general, `to` can tell us what those unkown keyword args for `f`.

### Additional features with `keep` and `but`
#### When to use `keep=False`?

`f` is happy with all the params it got, no more expectation of other kwargs, so remove `**kwargs` from its signature.

#### when to use `keep=True`?

`f` is not content with all the params it got, and still expect other kwargs in the future when needed.

#### when to use `but=['d']`?

`f` does not want a list of specified kwargs from `to`, such as `d` here.

### Creating your own examples to test the boundary of usages

In [ ]:
def low(a, b=1, c=1, **kwargs): pass # to receive unexpected kwargs, `to` need `**kwargs` too.
@delegates(low, keep=True, but=['b'])
def mid(a, d=1, **kwargs): pass
test_sig(mid, '(a, d=1, *, c=1, **kwargs)')
mid(a=1, d=1, c=1, e=1) # mid is open to other unexpected kwargs without error

In [ ]:
def low(a, b=1): pass 
@delegates(low) # to as func
def mid(c, d=1, **kwargs): pass # f as func
test_sig(mid, '(c, d=1, *, b=1)')

In [ ]:
class Foo():

    @delegates(low) # to as func
    @classmethod # classmethod() run before delegates()
    def clsmid1(cls, c, d=1, **kwargs): pass # f as classmethod


    @delegates(clsmid1) # to as classmethod
    @classmethod
    def clsmid2(cls, c, e=1, **kwargs): pass # f as classmethod

test_sig(Foo.clsmid1, '(c, d=1, *, b=1)')
test_sig(Foo.clsmid2, '(c, e=1, *, d=1, b=1)')

In [ ]:
class Foo():
    
	@classmethod
	@delegates(low) # to as func
	def clsmid1(cls, c, d=1, **kwargs): pass # f as func, not classmethod

	@classmethod 
	@delegates(clsmid1)  # to as classmethod 
	def clsmid2(cls, c, e=1, **kwargs): pass # f as func, not classmethod

test_sig(Foo.clsmid1, '(c, d=1, *, b=1)')
test_sig(Foo.clsmid2, '(c, e=1, *, d=1, b=1)')

In [ ]:
class Foo1():

    @delegates(Foo.clsmid2) # to as classmethod
    def instmid1(self, c, f=1, **kwargs): pass # f as func

    @delegates(instmid1) # to as func type
    def instmid2(self, c, g=1, **kwargs): pass # f as func 

f = Foo1()
test_sig(f.instmid1, '(c, f=1, *, e=1, d=1, b=1)')
test_sig(f.instmid2, '(c, g=1, *, f=1, e=1, d=1, b=1)')

In [ ]:
class Foo():
    @classmethod
    def f(cls, a=1, c=2): pass

@delegates(Foo.f) # `to` is a classmethod
def low(a, b=1, **kwargs): pass # `f` is a function
test_sig(low, '(a, b=1, *, c=2)')

In [ ]:
class Foo():
	def __init__(a, b=1, **kwargs): pass

@delegates(Foo) # `to` is a class
def low(c, d=1, **kwargs): pass # `f` as function

test_sig(low, '(c, d=1, *, b=1)')

In [ ]:
class Base(): 
    def __init__(self, a, b=1): pass 
class Other():
    def __init__(self, e=1): pass

@delegates(Other)
class Foo():    
    def __new__(self, g, f=1, **kwargs): pass # f as a class is ok with either __new__ or __init__
test_sig(Foo, '(g, f=1, *, e=1)')

In [ ]:
@delegates()
class Subcls(Base): # to as a superclass (Base),  to must have __init__, and __new__ won't do. 
    ### why?
    # As source code required to use Base.__init__, if it doesn't have it, it will use a wrap-slot __init__, which has no __annotations__, and will
    # cause error when __annotations__ is called later.
    def __init__(self, c, d=1, **kwargs): pass # f as class, should have __init__
test_sig(Subcls, '(c, d=1, *, b=1)')

In [ ]:
@delegates(Other) # as a normal class, it must have __init__, and __new__ won't do
class Foo():    
    def __init__(self, c, d=1, **kwargs): pass
test_sig(Foo, '(c, d=1, *, e=1)')

In [ ]:
try: 
    @delegates(Other)
    class Foo(): pass # f as a class at least needs to have a signature with **kwargs inside
    ### why? because we expect f to have **kwargs in the source.
except: 
    pprint(inspect.signature(Foo))

<Signature (c, d=1, *, e=1)>


### debugging the error to understand the source

## Annotation Source

In [ ]:
delegates??

Signature: delegates(to: function = None, keep=False, but: list = None)
Source:   
def delegates(to:FunctionType=None, # Delegatee
              keep=False, # Keep `kwargs` in decorated function?
              but:list=None): # Exclude these parameters from signature
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
      

In [ ]:
def delegatesdb(to:FunctionType=None, # Delegatee
              keep=False, # Keep `kwargs` in decorated function?
              but:list=None): # Exclude these parameters from signature
              # verbose=True): # Include `to` in docments?
    "Decorator: replace `**kwargs` in signature with params from `to`"
    
    "\nWhen `f` overwrite `to` (or simply just want to use\
    `to`'s kwargs), it's so convenient to keep `to`'s kwargs inside `f`'s signature instead of just `**kwargs`."
    
    if but is None: but = []
    def _f(f):
        
        block1 = """
if to is None: to_f,from_f = f.__base__.__init__,f.__init__
else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        """
        dbprint(block1, "if to is None: when both to and f are classes\nif isinstance(to, type): when to is a class", block1, "to_f", "from_f", \
                "\nif to is None:\n    to_f,from_f = f.__base__.__init__,f.__init__\n    print(f'f.__base__.__init__: {f.__base__.__init__}, f.__init__: {f.__init__}')\n    print(f'to_f: {to_f}, from_f: {from_f}')\nelse:\n    print(f'to is None : {to is None}')", \
                "\nif isinstance(to,type):\n    to_f,from_f = to.__init__,f\n    print(f'to_f: {to_f}, from_f: {from_f}')\nelse:\n    print(f'isinstance(to,type) : {isinstance(to,type)}')",\
                "to_f,from_f = to,f", f=f, to=to)
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f

        dbprint("from_f = getattr(from_f,'__func__',from_f)", "This line is for classmethod, as it is not callable, so inspect.signature(...) won't work, but it has __func__ to save",\
                "f", "from_f", "type(from_f)",\
                "try:\n    inspect.signature(from_f)\nexcept:\n    print('error occurs')\n    print(f'is from_f callable: {callable(from_f)}')\nelse:\n    print(inspect.signature(from_f))", \
                "hasattr(from_f, '__func__')", "from_f = getattr(from_f,'__func__',from_f)", "from_f", from_f=from_f, inspect=inspect)
        from_f = getattr(from_f,'__func__',from_f)
        
        dbprint("to_f = getattr(to_f,'__func__',to_f)", "This line is for classmethod, as it is not callable, so inspect.signature(...) won't work, but it has __func__ to save", \
                "to", "to_f", "type(to_f)",\
                "try:\n    inspect.signature(to_f)\nexcept:\n    print('error occurs')\n    print(f'is to_f callable: {callable(to_f)}')\nelse:\n    print(inspect.signature(to_f))", \
                "hasattr(to_f, '__func__')", "to_f = getattr(to_f,'__func__',to_f)", "to_f", to_f=to_f)
        to_f = getattr(to_f,'__func__',to_f)
        
        dbprint("if hasattr(from_f,'__delwrap__'): return f", "If f or from_f has __delwrap__, it means it's happy with all params and give up on **kwargs. So no more params needed from other `to`s",\
                "from_f", "f", "hasattr(from_f, '__delwrap__')", "if hasattr(from_f,'__delwrap__'): return f",  from_f=from_f, to_f=to_f)
        # dbprint("if hasattr(from_f,'__delwrap__'): return f", "If f or from_f has __delwrap__, it means it's happy with all params and give up on **kwargs. So no more params needed from other `to`s",\
        #         "from_f", "f", "hasattr(from_f, '__delwrap__')",  from_f=from_f, to_f=to_f)
        if hasattr(from_f,'__delwrap__'): return f

        dbprint("sig = inspect.signature(from_f)", "", "sig = inspect.signature(from_f)", "inspect.signature(from_f)", inspect=inspect)
        sig = inspect.signature(from_f)
        
        dbprint("sigd = dict(sig.parameters)", "", "sigd = dict(sig.parameters)", "sig.parameters", "dict(sig.parameters)", sig=sig)
        sigd = dict(sig.parameters)
        
        # remove the item {'kwargs': <Parameter "**kwargs">} out of sigd
        dbprint("k = sigd.pop('kwargs')", "", "sigd", "k = sigd.pop('kwargs')", "sigd", "sigd['kwargs'] = k", sigd=sigd)
        k = sigd.pop('kwargs')
              
        # build a dict and pick the right keyword args with values into it
        
        block2 = """s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()                                         
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}"""
        dbprint(block2, "", block2, "s2", "for k,v in inspect.signature(to_f).parameters.items():", "sigd", "but", \
                "for k,v in inspect.signature(to_f).parameters.items():\n    print(f'v.default: {v.default}')\n    print(f'v.default != inspect.Parameter.empty: {v.default != inspect.Parameter.empty}')\n    print(f'k not in sigd: {k not in sigd}')\n    print(f'k not in but: {k not in but}')", \
                to_f=to_f, but=but)
        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        
        dbprint("anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}", "", \
                "anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}", "anno",\
                "for k,v in to_f.__annotations__.items():", "sigd", "but")
        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}     
        
        dbprint("sigd.update(s2)", "", "s2", "sigd", "sigd.update(s2)", "sigd", s2=s2)
        sigd.update(s2)                                                   
        
        block3 = """
if keep: sigd['kwargs'] = k
else: from_f.__delwrap__ = to_f
        """
        dbprint(block3, "", "k", "keep", "sigd", "to_f", "getattr(from_f, '__delwrap__', None)", \
                "if keep:\n    sigd['kwargs'] = k\nelse:\n    from_f.__delwrap__ = to_f", "sigd", "from_f.__delwrap__", k=k, keep=keep)
        if keep: sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        
        dbprint("from_f.__signature__ = sig.replace(parameters=sigd.values())", "", "getattr(from_f, '__signature__', None)", "sig", "sigd.values()", \
               "inspect.getdoc(sig.replace)", "from_f.__signature__ = sig.replace(parameters=sigd.values())", \
                "getattr(from_f, '__signature__', None)", sigd=sigd)
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        
        dbprint("if hasattr(from_f, '__annotations__'): from_f.__annotations__.update(anno)", "", "anno", "getattr(from_f, '__annotations__', None)", \
                "if hasattr(from_f, '__annotations__'): from_f.__annotations__.update(anno)", "getattr(from_f, '__annotations__', None)", from_f=from_f, anno=anno)
        if hasattr(from_f, '__annotations__'): from_f.__annotations__.update(anno)                                                                           
        return f                                                                                                                                             
    return _f       


## Add srcline and dbprints

In [ ]:
# srcdbps = defaults.src2dbp.delegates # user input
srcdbps = []

In [ ]:
dbsrclines(srcname) # to show which lines have dbcodes already

def delegates(to:FunctionType=None, # Delegatee                                                                                                              
              keep=False, # Keep `kwargs` in decorated function?                                                                                             
              but:list=None): # Exclude these parameters from signature                                                                                      
    "Decorator: replace `**kwargs` in signature with params from `to`"                                                                                       
    if but is None: but = []                                                                                                                                 
    def _f(f):                                                                                                                                               
        if to is None: to_f,from_f = f.__base__.__in

In [ ]:
srcline = "        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}"
dbcode = """
        dbprint("        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}", \
"what it does: check to_f's annotations (dict), and only select params with their annotations wanted by f or from_f.", \
"hasattr(to_f, '__annotations__')", \
"if hasattr(to_f, '__annotations__'):\\n\
    for k,v in to_f.__annotations__.items():\\n\
        print(f'k:v => {k}:{v}')\\n\
else:\\n\
    print(f'We should expect error from the code of try-except.')\\n\
    try:\\n\
        to_f.__annotations__\\n\
    except AttributeError as e:\\n\
        print(e)", \
but=but, k=k, sigd=sigd, to_f=to_f)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = """
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
"""

# watch out: to convert srcline into actual strings for the first arg in dbprint, use '\\n\' instead of just '\n'
dbcode = """
\n        dbprint("        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\\n\        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f",\
"UseCase1: when to is superclass to f\\nUseCase2: when to is a class but not superclass to f. UseCase3: other combinations",\
"if to is None:\\n\
    to_f,from_f = f.__base__.__init__,f.__init__\\n\
else:\\n\
    to_f,from_f = to.__init__ if isinstance(to,type) else to,f",\
"to_f", "from_f", "hasattr(to_f, '__annotations__')", "hasattr(from_f, '__annotations__')", \
f=f, to=to)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = "        if hasattr(from_f,\'__delwrap__\'): return f"
dbcode = """
        dbprint("if hasattr(from_f,'__delwrap__'): return f", \
"If f or from_f has __delwrap__, it means it's happy with all params and give up on **kwargs. So no more params needed from other `to`s", \
"from_f", "f", "hasattr(from_f, '__delwrap__')", "if hasattr(from_f,'__delwrap__'): return f", from_f=from_f, f=f)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
# Watch out: using \\n instead of \n inside a block
srcline = """
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
"""
dbcode = """
\n        dbprint("        from_f = getattr(from_f,'__func__',from_f)\\n\        to_f = getattr(to_f,'__func__',to_f)",\
"This line is for classmethod, as it is not callable, so inspect.signature(...) won't work, but it has __func__ to save",\
"f", "from_f", "type(from_f)",\
"try:\\n\
    inspect.signature(from_f)\\n\
except:\\n\
    print('error occurs')\\n\
    print(f'is from_f callable: {callable(from_f)}')\\n\
else:\\n\
    print(inspect.signature(from_f))", \
"hasattr(from_f, '__func__')", "from_f = getattr(from_f,'__func__',from_f)", "from_f",\
"to", "to_f", "type(to_f)",\
"try:\\n\
    inspect.signature(to_f)\\n\
except:\\n\
    print('error occurs')\\n\
    print(f'is to_f callable: {callable(to_f)}')\\n\
else:\\n\
    print(inspect.signature(to_f))", \
"hasattr(to_f, '__func__')", "to_f = getattr(to_f,'__func__',to_f)", "to_f",\
from_f=from_f, to_f=to_f, f=f, to=to, srcline=srcline)
"""
# Importance: must include all necessary env into the dbprint function above. Note, we don't need inspect=inspect because utils has exported `import inspect`

In [ ]:
srcdbps.append([(srcline, dbcode)])

## Save the debuggable source code

In [ ]:
# save all the srclines with their dbcodes into defaults.
defaults.src2dbp.delegates = srcdbps

In [ ]:
# save the latest debuggable source code into defaults.src2dbp.delegatesdb
dbsrclines(srcname, dbsrc=True) 

In [ ]:
file1 = open("db/delegates.txt","w")
file1.writelines(defaults.src2dbp.delegatesdb)
file1.close() #to change file access modes  

## Debugging one or more srclines

In [ ]:
dbsrclines(srcname) # check which srclines are debuggable

def delegates(to:FunctionType=None, # Delegatee                                                                                                              
              keep=False, # Keep `kwargs` in decorated function?                                                                                             
              but:list=None): # Exclude these parameters from signature                                                                                      
    "Decorator: replace `**kwargs` in signature with params from `to`"                                                                                       
    if but is None: but = []                                                                                                                                 
    def _f(f):                                                                                                                                               
        if to is None: to_f,from_f = f.__base__.__in

In [ ]:
delegates = dbsrclines(srcname, [0,1,2,3]) # print out the debuggable source code under investgation

('def delegates(to:FunctionType=None, # Delegatee\n'
 '              keep=False, # Keep `kwargs` in decorated function?\n'
 '              but:list=None): # Exclude these parameters from signature\n'
 '    "Decorator: replace `**kwargs` in signature with params from `to`"\n'
 '    if but is None: but = []\n'
 '    def _f(f):\n'
 '\n'
 '        dbprint("        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\\n\\        else:          to_f,from_f = to.__init__ if '
 'isinstance(to,type) else to,f","UseCase1: when to is superclass to f\\nUseCase2: when to is a class but not superclass to f. UseCase3: other '
 'combinations","if to is None:\\n    to_f,from_f = f.__base__.__init__,f.__init__\\nelse:\\n    to_f,from_f = to.__init__ if isinstance(to,type) else '
 'to,f","to_f", "from_f", "hasattr(to_f, \'__annotations__\')", "hasattr(from_f, \'__annotations__\')", f=f, to=to)\n'
 '\n'
 '        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\n'
 '        else:        

In [ ]:
defaults.eg = """
class Base(): 
    def __init__(self, a, b=1): pass 
class Other():
    def __init__(self, e=1): pass

@delegates()
class Subcls(Base): # to as a superclass (Base),  to must have __init__, and __new__ won't do
    def __init__(self, c, d=1, **kwargs): pass # f as class, should have __init__
test_sig(Subcls, '(c, d=1, *, b=1)')
"""

class Base(): 
    def __init__(self, a, b=1): pass 
class Other():
    def __init__(self, e=1): pass

@delegates()
class Subcls(Base): # to as a superclass (Base),  to must have __init__, and __new__ won't do
    def __init__(self, c, d=1, **kwargs): pass # f as class, should have __init__
test_sig(Subcls, '(c, d=1, *, b=1)')



######################################################## source code with lines under investigation #########################################################


def delegates(to:FunctionType=None, # Delegatee                                                                                                              
              keep=False, # Keep `kwargs` in decorated function?                                                                                             
              but:list=None): # Exclude these parameters from signature                                                                                      
    "Decorator: replace `**kwargs` in signature with params from `to`"                                                                                       
    if but is None: but = []                                                                                                                                 
    def _f(f):                                  

#|hide
## Sending to Obs

In [ ]:
#|hide
!jupytext --to md /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.ipynb
!mv /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/fastcore/meta/

[jupytext] Reading /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.ipynb in format ipynb
[jupytext] Writing /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.md


In [ ]:
#|hide
!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert

[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/index.ipynb to markdown
[NbConvertApp] Writing 2074 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/index.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/utils.ipynb to markdown
[NbConvertApp] Writing 165629 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/utils.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/classes_metaclasses.ipynb to markdown
[NbConvertApp] Writing 23234 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/classes_metaclasses.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/meta/02_use_kwargs_dict.ipynb to markdown
[NbConvertApp] Writing 3777 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/02_use_kwargs_dict.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.ipynb to markdown
[NbConvertAp